In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import rb_equivariant_cnn as conv
import rb_equivariant_gcnn as gconv
import rb_equivariant_se2ncnn as dn_conv

2024-08-05 07:42:17.608146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Setup

In [2]:
RB_CHANNELS = 4
HORIZONTAL_SIZE = 64
HEIGHT = 32

BATCH_SIZE = 64

# Data Augmentation

In [3]:
# Rotate and Flip Vectors
class RandomRot(keras.layers.RandomRotation):
    def call(self, inputs: tf.Tensor, *args, **kwargs) -> tf.Tensor:
        in_shape = tf.shape(inputs)
        inputs = tf.reshape(inputs, in_shape[:-2]+[np.prod(in_shape[-2:])])
        outputs = super().call(inputs, *args, **kwargs)
        return tf.reshape(outputs, in_shape)
    
class RandomFlip(keras.layers.RandomFlip):
    def call(self, inputs: tf.Tensor, *args, **kwargs) -> tf.Tensor:
        in_shape = tf.shape(inputs)
        inputs = tf.reshape(inputs, in_shape[:-2]+[np.prod(in_shape[-2:])])
        outputs = super().call(inputs, *args, **kwargs)
        return tf.reshape(outputs, in_shape)

# 3D Rayleigh-Bénard Convolution
- Equivariant to horizontal translations
- __No vertical parameter sharing__
- Height dependend bias
- Supports horizontal wrap and same padding
    - Wrap makes sense when using peridoc boundary conditions for Rayleigh-Bénard
    - Attention: This may destroy exact rotation equivariance in our experiments (nevertheless WRAP will be preferable in practice)
- Also supports vertical same padding
- Supports stride (including vertical stride)
- Uses 2D convolutions under the hood

In [4]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            # Data Augmentation
            RandomRot(factor=1, fill_mode='wrap', value_range=(0,1)),
            RandomFlip(mode='horizontal_and_vertical'),
            
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv1'),
            conv.BatchNorm(name='BatchNorm1'),
            keras.layers.Activation('relu', name='NonLinearity1'),
            
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv2'),
            conv.BatchNorm(name='BatchNorm2'),
            keras.layers.Activation('relu', name='NonLinearity2'),
            
            conv.SpatialPooling(ksize=(2,2,2), strides=(2,2,2), pooling_type='MAX'),
            
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv3'),
            conv.BatchNorm(name='BatchNorm3'),
            keras.layers.Activation('relu', name='NonLinearity3'),
            
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv4'),
            conv.BatchNorm(name='BatchNorm4'),
            keras.layers.Activation('relu', name='NonLinearity4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_rot (RandomRot)          │ (64, 64, 64, 32, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (64, 64, 64, 32, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 (RB3D_Conv)               │ (64, 33, 33, 17, 4)    │        12,308 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm1 (BatchNorm)          │ (64, 33, 33, 17, 4)    │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity1 (Activation)      │ (64, 33, 33, 17, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (64, 33, 33, 17, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (RB3D_Conv)               │ (64, 17, 17, 9, 4)     │         6,516 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm2 (BatchNorm)          │ (64, 17, 17, 9, 4)     │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity2 (Activation)      │ (64, 17, 17, 9, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling (SpatialPooling) │ (64, 8, 8, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (64, 8, 8, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv3 (RB3D_Conv)               │ (64, 5, 5, 3, 4)       │         2,172 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm3 (BatchNorm)          │ (64, 5, 5, 3, 4)       │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity3 (Activation)      │ (64, 5, 5, 3, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (64, 5, 5, 3, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv4 (RB3D_Conv)               │ (64, 3, 3, 2, 4)       │         1,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm4 (BatchNorm)          │ (64, 3, 3, 2, 4)       │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity4 (Activation)      │ (64, 3, 3, 2, 4)       │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,940 (89.61 KB)

 Trainable params: 22,692 (88.64 KB)

 Non-trainable params: 248 (992.00 B)

# 3D Rayleigh-Bénard $D_4$ Group Equivariant Convolution
- Equivariant to all symmetries of 3D Rayleigh-Bénard:
    - __90° rotations around a vertical axis__
    - __reflections through a vertical plane__
    - __horizontal translations__

In [5]:
G = 'D4' # 'C4' for rotations or 'D4' for rotations and reflections
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            gconv.RB3D_G_Conv('Z2', G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'Lift_{G}_Conv1'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv2'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv3'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (64, 64, 64, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Lift_D4_Conv1 (RB3D_G_Conv)     │ (64, 33, 33, 8, 17, 4) │        12,308 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv2 (RB3D_G_Conv)          │ (64, 17, 17, 8, 9, 4)  │        51,876 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv3 (RB3D_G_Conv)          │ (64, 9, 9, 8, 5, 4)    │        28,820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv4 (RB3D_G_Conv)          │ (64, 5, 5, 8, 3, 4)    │        17,292 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,296 (430.84 KB)

 Trainable params: 110,296 (430.84 KB)

 Non-trainable params: 0 (0.00 B)

# 3D Rayleigh-Bénard $D_N$ Group Equivariant Convolution
- Equivariant to all symmetries of 3D Rayleigh-Bénard:
    - __arbitrary discrete__ rotations around a vertical axis
    - reflections through a vertical plane
    - horizontal translations

In [6]:
ORIENTATIONS = 8

model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            dn_conv.RB3D_LiftDN_Conv(orientations=ORIENTATIONS, h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP',
                                     v_padding='SAME', strides=(2, 2, 2), name='Lift_DN_Conv1'),
            dn_conv.RB3D_DN_Conv(h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', 
                                 v_padding='SAME', strides=(2, 2, 2), name='DN_Conv2'),
            dn_conv.RB3D_DN_Conv(h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', 
                                 v_padding='SAME', strides=(2, 2, 2), name='DN_Conv3'),
            dn_conv.RB3D_DN_Conv(h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', 
                                 v_padding='SAME', strides=(2, 2, 2), name='DN_Conv4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Lift_DN_Conv1                   │ (64, 33, 33, 16, 17,   │        34,068 │
│ (RB3D_LiftDN_Conv)              │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DN_Conv2 (RB3D_DN_Conv)         │ (64, 17, 17, 16, 9, 4) │       288,036 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DN_Conv3 (RB3D_DN_Conv)         │ (64, 9, 9, 16, 5, 4)   │       160,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DN_Conv4 (RB3D_DN_Conv)         │ (64, 5, 5, 16, 3, 4)   │        96,012 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 578,136 (2.21 MB)

 Trainable params: 578,136 (2.21 MB)

 Non-trainable params: 0 (0.00 B)

# Autoencoder

#### Convolutional Autoencoder

In [7]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            ###############
            #   Encoder   #
            ###############
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_Conv1'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool1'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_Conv2'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool2'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_Conv3'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool3'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_Conv4'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='Pool4'),
            
            ###############
            #   Decoder   #
            ###############
            conv.UpSampling(size=(2,2,2), name='UpSampling1'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_Conv1'),
            conv.UpSampling(size=(2,2,2), name='UpSampling2'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_Conv2'),
            conv.UpSampling(size=(2,2,2), name='UpSampling3'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_Conv3'),
            conv.UpSampling(size=(2,2,2), name='UpSampling4'),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_Conv4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ En_Conv1 (RB3D_Conv)            │ (64, 64, 64, 32, 4)    │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool1 (SpatialPooling)          │ (64, 32, 32, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_Conv2 (RB3D_Conv)            │ (64, 32, 32, 16, 4)    │        11,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool2 (SpatialPooling)          │ (64, 16, 16, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_Conv3 (RB3D_Conv)            │ (64, 16, 16, 8, 4)     │         5,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool3 (SpatialPooling)          │ (64, 8, 8, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_Conv4 (RB3D_Conv)            │ (64, 8, 8, 4, 4)       │         2,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool4 (SpatialPooling)          │ (64, 4, 4, 2, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling1 (UpSampling3D)      │ (64, 8, 8, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_Conv1 (RB3D_Conv)            │ (64, 8, 8, 4, 4)       │         2,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling2 (UpSampling3D)      │ (64, 16, 16, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_Conv2 (RB3D_Conv)            │ (64, 16, 16, 8, 4)     │         5,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling3 (UpSampling3D)      │ (64, 32, 32, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_Conv3 (RB3D_Conv)            │ (64, 32, 32, 16, 4)    │        11,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling4 (UpSampling3D)      │ (64, 64, 64, 32, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_Conv4 (RB3D_Conv)            │ (64, 64, 64, 32, 4)    │        23,168 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,880 (339.38 KB)

 Trainable params: 86,880 (339.38 KB)

 Non-trainable params: 0 (0.00 B)

### $D_4$ Group Equivariant Convolutional Autoencoder

In [8]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            ###############
            #   Encoder   #
            ###############
            gconv.RB3D_G_Conv('Z2', G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), 
                              name=f'En_Lift_{G}_Conv1'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool1'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), 
                              name=f'En_{G}-Conv2'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool2'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              name=f'En_{G}-Conv3'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool3'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              name=f'En_{G}-Conv4'),
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool4'),
            
            ###############
            #   Decoder   #
            ###############
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), 
                              name=f'De_{G}_Conv1'),
            gconv.UpSampling(size=(2,2,2), name='UpSampling1'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1), 
                              name=f'De_{G}-Conv2'),
            gconv.UpSampling(size=(2,2,2), name='UpSampling2'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              name=f'De_{G}-Conv3'),
            gconv.UpSampling(size=(2,2,2), name='UpSampling3'),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              name=f'De_{G}-Conv4'),
            gconv.UpSampling(size=(2,2,2), name='UpSampling4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)             │ (64, 64, 64, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_Lift_D4_Conv1 (RB3D_G_Conv)  │ (64, 64, 64, 8, 32, 4) │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool1 (SpatialPooling)   │ (64, 32, 32, 8, 16, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_D4-Conv2 (RB3D_G_Conv)       │ (64, 32, 32, 8, 16, 4) │        92,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool2 (SpatialPooling)   │ (64, 16, 16, 8, 8, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_D4-Conv3 (RB3D_G_Conv)       │ (64, 16, 16, 8, 8, 4)  │        46,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool3 (SpatialPooling)   │ (64, 8, 8, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_D4-Conv4 (RB3D_G_Conv)       │ (64, 8, 8, 8, 4, 4)    │        23,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool4 (SpatialPooling)   │ (64, 4, 4, 8, 2, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_D4_Conv1 (RB3D_G_Conv)       │ (64, 4, 4, 8, 2, 4)    │        11,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling1 (UpSampling)        │ (64, 8, 8, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_D4-Conv2 (RB3D_G_Conv)       │ (64, 8, 8, 8, 4, 4)    │        23,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling2 (UpSampling)        │ (64, 16, 16, 8, 8, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_D4-Conv3 (RB3D_G_Conv)       │ (64, 16, 16, 8, 8, 4)  │        46,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling3 (UpSampling)        │ (64, 32, 32, 8, 16, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_D4-Conv4 (RB3D_G_Conv)       │ (64, 32, 32, 8, 16, 4) │        92,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling4 (UpSampling)        │ (64, 64, 64, 8, 32, 4) │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 357,480 (1.36 MB)

 Trainable params: 357,480 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

### $D_N$ Group Equivariant Convolutional Autoencoder

In [9]:
ORIENTATIONS = 8

model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            ###############
            #   Encoder   #
            ###############
            dn_conv.RB3D_LiftDN_Conv(orientations=ORIENTATIONS, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                     h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_Lift_DN_Conv1'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool1'),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_DN-Conv2'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool2'),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_DN-Conv3'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool3'),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='En_DN-Conv4'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID', name='SpatialPool4'),
            
            ###############
            #   Decoder   #
            ###############
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_DN_Conv1'),
            dn_conv.UpSampling(size=(2,2,2), name='UpSampling1'),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_DN-Conv2'),
            dn_conv.UpSampling(size=(2,2,2), name='UpSampling2'),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_DN-Conv3'),
            dn_conv.UpSampling(size=(2,2,2), name='UpSampling3'),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, 
                                 h_padding='WRAP', v_padding='SAME', strides=(1,1,1), name='De_DN-Conv4'),
            dn_conv.UpSampling(size=(2,2,2), name='UpSampling4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ En_Lift_DN_Conv1                │ (64, 64, 64, 16, 32,   │        23,168 │
│ (RB3D_LiftDN_Conv)              │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool1 (SpatialPooling)   │ (64, 32, 32, 16, 16,   │             0 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_DN-Conv2 (RB3D_DN_Conv)      │ (64, 32, 32, 16, 16,   │       184,384 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool2 (SpatialPooling)   │ (64, 16, 16, 16, 8, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_DN-Conv3 (RB3D_DN_Conv)      │ (64, 16, 16, 16, 8, 4) │        92,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool3 (SpatialPooling)   │ (64, 8, 8, 16, 4, 4)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ En_DN-Conv4 (RB3D_DN_Conv)      │ (64, 8, 8, 16, 4, 4)   │        46,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPool4 (SpatialPooling)   │ (64, 4, 4, 16, 2, 4)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_DN_Conv1 (RB3D_DN_Conv)      │ (64, 4, 4, 16, 2, 4)   │        23,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling1 (UpSampling)        │ (64, 8, 8, 16, 4, 4)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_DN-Conv2 (RB3D_DN_Conv)      │ (64, 8, 8, 16, 4, 4)   │        46,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling2 (UpSampling)        │ (64, 16, 16, 16, 8, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_DN-Conv3 (RB3D_DN_Conv)      │ (64, 16, 16, 16, 8, 4) │        92,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling3 (UpSampling)        │ (64, 32, 32, 16, 16,   │             0 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ De_DN-Conv4 (RB3D_DN_Conv)      │ (64, 32, 32, 16, 16,   │       184,384 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling4 (UpSampling)        │ (64, 64, 64, 16, 32,   │             0 │
│                                 │ 4)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691,560 (2.64 MB)

 Trainable params: 691,560 (2.64 MB)

 Non-trainable params: 0 (0.00 B)